In [32]:
######################################################################
# Minter
# 
# Scrapes twitter & Opensea for newest freemints and mints them
######################################################################


import pandas as pd
from web3 import Web3
import requests
import time
import snscrape.modules.twitter as sntwitter
import os
import time
import datetime
import pytz
from bs4 import BeautifulSoup
import json
import random
from scrapingbee import ScrapingBeeClient
# import cloudscraper

import dotenv
dotenv.load_dotenv("../../.env")



# scraper = cloudscraper.create_scraper() # doesnt work, cloudflare :(
client = ScrapingBeeClient(api_key=os.getenv("SCRAPINGBEE_API_KEY"))

utc=pytz.UTC

web3 = Web3(Web3.HTTPProvider(os.getenv("INFURA_ETHEREUM_MAINNET_URI")))
admin_account = web3.eth.account.from_key(os.getenv("WALLET_KEY"))


user_agents = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2919.83 Safari/537.36',
]

def get_tweets(account="@FreeMintsAlert", mins=5, n=10):
    # gets all tweets from account in last n mins

    tweets = []
    query = f'from:{account}'
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
        if i >= n:
            break
        if utc.localize(datetime.datetime.utcnow()) - tweet.date < datetime.timedelta(minutes=mins):
            tweets.append(tweet)
        # else:
        #     print(f"tweet {i} is too old, date: {tweet.date}. delta: {utc.localize(datetime.datetime.utcnow()) - tweet.date}")
    return tweets


def unshorten(twitter_url):
    response = requests.get(twitter_url)
    return response.url


def get_contract_address(opensea_url):
    # loads opensea website and gets contract address
    # scraper = cloudscraper.create_scraper()
    # scraper.headers['User-Agent'] = random.choice(user_agents)
    # response = scraper.get(opensea_url)
    # response = requests.get(opensea_url, headers=headers)
    response = client.get(opensea_url)

    if response.status_code != 200:
        print(f"response code: {response.status_code}")
        # save html to file (with timestamp)
        file_name = f"logs/{datetime.datetime.utcnow().timestamp()}.html"
        with open(file_name, "w") as f:
            f.write(response.text)
            print(f"saved response html to {file_name}. Requested URL was: {opensea_url}")
        
        raise Exception("response code not 200")

    # save html to file
    with open('opensea.html', 'w') as f:
        f.write(response.text)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = []
    # find any link that starts with https://etherscan.io/address/
    for link in soup.find_all('a'):
        href = link.get('href')
        if href is not None and href.startswith('https://etherscan.io/address/'):
            links.append(href)
    
    assert len(links) > 0, "no links found"
    assert len(links) < 2, "More than one contract address found"

    contract_address = links[0].split('/')[-1]
    return contract_address


def get_abi(contract_address):
    # gets abi from etherscan api
    url = f'https://api.etherscan.io/api?module=contract&action=getabi&address={contract_address}&apikey={os.getenv("ETHERSCAN_KEY_OWN")}'
    response = requests.get(url)
    abi = response.json()['result']
    abi = json.loads(abi)
    return abi


# get current price of a token
def get_price(token):
    url = "https://api.binance.com/api/v3/ticker/price"
    params = {"symbol": token + "USDT"}
    response = requests.get(url, params=params)
    data = response.json()
    price = data["price"]
    return price


def mint(contract_address, abi):
    contract_address = Web3.toChecksumAddress(contract_address)
    # mints a token from contract_address using 'mint' or 'freemint' function
    contract = web3.eth.contract(address=contract_address, abi=abi)
    nonce = web3.eth.getTransactionCount(admin_account.address)

    # check that we have 0 balance in contract (havent minted yet)
    balance = contract.functions.balanceOf(admin_account.address).call()
    assert balance == 0, "balance is not 0"
    
    gas_price = web3.eth.gasPrice # LEGACY
    print(f"gas price: {gas_price/1e9} gwei")
    # if gasprice bigger than 20 gwei, return
    # if gas_price > 40e9:
    #     print(f"gas price too high: {gas_price/1e9} gwei")
    #     return

    mint_function = None
    for i in abi:
        if i['type'] == 'function':
            if i['name'] == 'mint':
                mint_function = i
                break
    
    if mint_function is None:
        print('no mint function found')
        return
    # get number of parameters
    n_params = len(mint_function['inputs'])

    # if 1 param and internalType is uint256, then mint 1
    if not (n_params == 1 and mint_function['inputs'][0]['internalType'] == 'uint256'):
        print(f"can't figure out how to mint, returning")
        return

    # build transaction with EIP-1559    
    max_priority_fee = web3.eth.max_priority_fee
    assert max_priority_fee < 4e9, f"max priority fee too high: {max_priority_fee/1e9} gwei"
    
    txn = contract.functions.mint(1).buildTransaction({
        'chainId': 1,
        'gas': 100000,
        # 'gasPrice': gas_price, # dont need this if using EIP-1559
        'maxPriorityFeePerGas': max_priority_fee,
        'maxFeePerGas': gas_price,
        'nonce': nonce,
        'from': admin_account.address,
    })

    eth_price = int(float(get_price("ETH"))) # python can use bignum with ints
    gas_estimate = web3.eth.estimateGas(txn)
    cost = gas_estimate * gas_price * eth_price / 1e18
    print(f"MAX gas estimate in USD: ${cost}")

    # sign transaction
    signed_txn = admin_account.sign_transaction(txn)

    # send transaction
    tx_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
    print(f"tx hash: {tx_hash.hex()}")

    tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    # print how much it cost
    gas_used = tx_receipt['gasUsed']
    cost = gas_used * gas_price * eth_price / 1e18
    print(f"gas used: {gas_used}, TX cost in USD: ${cost}")

    # check if minted
    balance = contract.functions.balanceOf(admin_account.address).call()
    print(f"balance: {balance}")

    return tx_receipt


# check if a function containing 'mint' exists in abi
def has_mint_function(abi, debug=False):
    has_mint_flag = False
    
    for i in abi:
        if i['type'] == 'function':
            if 'mint' in i['name']:
                if debug:
                    print(i)
                has_mint_flag = True
    
    return has_mint_flag


def process_tweet(tweet):

    print(f"processing tweet: {tweet.url}")
    
    # find urls
    content = tweet.content
    urls = [url for url in content.split(' ') if url.startswith('https://t.co/')]
    urls = [unshorten(url) for url in urls]


    # find the url with 'opensea' in it and get contract address
    opensea_urls = [url for url in urls if 'opensea' in url]
    assert len(opensea_urls) > 0, f"No opensea url found. Opensea link: {opensea_urls}"
    assert len(opensea_urls) < 2, f"More than one opensea url found: {opensea_urls}"
    opensea_url = opensea_urls[0]
    contract_address = get_contract_address(opensea_url)

    # get abi & write to abis/contract_address_abi.json
    abi = get_abi(contract_address)
    # turn to dict
    with open(f'abis/{contract_address}_abi.json', 'w') as f:
        json.dump(abi, f, indent=4)
    print(f"saved abi to abis/{contract_address}_abi.json")

    return contract_address, abi
    # mint(contract_address, abi)


def run_once():
    # runs once and tries to mint on latest tweet
    tweets = get_tweets()


In [33]:
mins, n = 600, 10
tweets = get_tweets(mins=mins, n=n)
print(f"Found {len(tweets)} tweets in last {mins} mins")

Found 1 tweets in last 600 mins


In [26]:
contract_address, abi = process_tweet(tweets[0])
contract_address

processing tweet: https://twitter.com/FreeMintsAlert/status/1568227735939473408
saved abi to abis/0xb2caf20cbf903184287814e6bef4b3a503daf310_abi.json


'0xb2caf20cbf903184287814e6bef4b3a503daf310'

In [29]:
try:
    mint(contract_address, abi)
except Exception as e:
    print(e)
    print("-- ERROR: minting --")

gas price: 22.449955359 gwei
execution reverted: Max supply exceeded!
error minting


In [141]:
has_mint_function(abi, debug=True)

False

In [56]:
for i, tweet in enumerate(tweets):
    if i < 12:
        continue
    try:
        contract_address, abi = process_tweet(tweet)
    except:
        print(f"Error processing tweet {tweet.url}")
        continue

saved abi to abis/0x3ee9412f2b49ccff312a080743d9bc4cb0773b27_abi.json
saved abi to abis/0xea930142c75104f998dbccffefcf2f7eefcf1616_abi.json
saved abi to abis/0xa7bcde2cf437cb1e180a0e9663f2c1df650e32da_abi.json
saved abi to abis/0x5d95c4c73e152727750df27ff7da1c7169430dd6_abi.json
saved abi to abis/0x9a985ea766b36e0f52a84408848b9981dc0a3b7b_abi.json
saved abi to abis/0xf8d3554c6e047b4bb719dbd41d05d04736734e4f_abi.json
saved abi to abis/0x57ede1f2f96bc095530fecbe5c9ba249884baad0_abi.json
saved abi to abis/0x12632d6e11c6bbc0c53f3e281ea675e5899a5df5_abi.json
saved abi to abis/0x9041c731ee866cfb76812d3358af2e1ab40fb9a9_abi.json
saved abi to abis/0xe86469b7a2e19ad5ab06c1ed68f4b70db1164b36_abi.json
saved abi to abis/0xe77e59e5d9db886b54ec609d0e0add13fe358fba_abi.json
saved abi to abis/0xe2ca7205e609dd383a6a83a7591f2f9182ab1b7b_abi.json
saved abi to abis/0x38ad34fdef977cece9b23123f4b033536a331938_abi.json
saved abi to abis/0xa51cee4dd9a4e254dd5b8923024de5aab050391d_abi.json
saved abi to abis/0x

In [66]:
# load all abis, and find all functions that have 'mint' in them
abis = os.listdir('abis')
mintfuncs = []
for contract_abi in abis:
    with open(f'abis/{contract_abi}', 'r') as f:
        abi = json.load(f)
    for i in abi:
        if i['type'] == 'function':
            if 'mint' in i['name']:

                mintfuncs.append(i      name'])
                if i['name'] == 'mint':
                    print(i)
                    print(len(i['inputs']))

print(mintfuncs)
print(len(mintfuncs))

{'inputs': [{'internalType': 'uint256', 'name': '_mintAmount', 'type': 'uint256'}], 'name': 'mint', 'outputs': [], 'stateMutability': 'payable', 'type': 'function'}
1
{'inputs': [{'internalType': 'uint256', 'name': '_mintAmount', 'type': 'uint256'}], 'name': 'mint', 'outputs': [], 'stateMutability': 'payable', 'type': 'function'}
1
{'inputs': [{'internalType': 'uint256', 'name': 'amount', 'type': 'uint256'}], 'name': 'mint', 'outputs': [], 'stateMutability': 'payable', 'type': 'function'}
1
{'inputs': [{'internalType': 'uint256', 'name': '_mintAmount', 'type': 'uint256'}], 'name': 'mint', 'outputs': [], 'stateMutability': 'payable', 'type': 'function'}
1
{'inputs': [{'internalType': 'uint16', 'name': '_numberOfTokens', 'type': 'uint16'}, {'internalType': 'bytes', 'name': '_signature', 'type': 'bytes'}], 'name': 'mint', 'outputs': [], 'stateMutability': 'payable', 'type': 'function'}
2
{'inputs': [{'internalType': 'uint256', 'name': 'amount', 'type': 'uint256'}], 'name': 'mint', 'output